## Fast RCNN

### Input

Input for Fast RCNN is an image and a list of regional proposals.  

We will treat each image as $x_i \in \mathbb{R}^{H,W,C}$ and regional proposal as a list of $[\cdots (r,c,h,w)_i \cdots ]$

### Model

![fastrcnn](fastrcnn.png)

### New Additions

### RoI Pooling Layer

**RoI pooling**, given non-uniform sized windows, output a fixed sized feature map.  

- Inputs:
    - Feature Maps (result of convolution/max pooling layers)
    - Region Proposals (N, 5), where N is number of proposals and each column represent: 
        - image id
        - top left corner coordninates (r, c)
        - height and width of regional proposal (h, w)
        
- Hyperparameter: 
    - (H, W): fixed sized output feature map (i.e. output dimension will be (H, W, F))
- Process
    - Pull out regional proposal:  ((r, c), (r + h, c), (r, w+c), (r+h, w+c))
    - Divide the grid into (H, W) sub-grids, each grid is (h/H, w/W) size
    - perform max pooling at each subgrid

### Improvement with Respect to RCNN

1. Incorporated Training in one algorithm:  (Is this good or bad?)
2. RoI (Region of Interest Pooling)
3. Same Feature Map for all proposed regions

## Faster RCNN

### Input

Input for Faster RCNN is an image.  No predefined regional proposal is needed as input.  

We will treat each image as $x_i \in \mathbb{R}^{H,W,C}$ 

### Model

![fasterrcnn](fasterrcnn.png)

### New Additions:

### Region Proposal Network

Input is an image and outputs a set of rectangular object proposals.

#### General Idea:

Slide a small network over the convolutional feature map output (from the last shared layer).  For each n x n spatial window, we will generate
1. k anchor boxes of predefined size
2. Cast n x n into a lower dimensional feature vector.  
3. The lower dimensional feature vector will be fed into two 1 x 1 (or fully connected) network, results in two output vectors
    - $\mathbb{R}^{2k}$ - which is the score on how likely is the item an object
    - $\mathbb{R}^{4k}$ - for each anchor (out of k), will output (r, c, h, w) for input into RoI network
    
*Note*: Each anchor is fixed size, every anchor will produce various (r, c, h, w)* outputs that may deviated from anchor.  

## Mask RCNN

### Input

Input for Fast RCNN is an image and a list of regional proposals.  

We will treat each image as $x_i \in \mathbb{R}^{H,W,C}$ and regional proposal as a list of $[\cdots (r,c,h,w)_i \cdots ]$